This is a notebook to recreate Figure 8 of [Lamb, Taylor & van Haasteren 2023](https://arxiv.org/abs/2303.15442).  This notebook calculates the effective number of pulsars

In [1]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format ='retina'

import numpy as np
import matplotlib.pyplot as plt
import glob
from chainconsumer import ChainConsumer
import pickle
from ceffyl import Ceffyl, models
from enterprise.signals import parameter, gp_priors as gpp
from enterprise_extensions.model_utils import get_tspan
from natsort import natsorted
import la_forge.core as co
from la_forge.slices import SlicesCore
from tqdm import tqdm

# ACCRE-specific import to load correct latex file
## COMMENT OUT AS REQUIRED
import os
os.environ["PATH"] += os.pathsep + '/home/lambwg/latex/bin/x86_64-linux' 

In [2]:
# setup default plotting code
plt.rcParams.update(plt.rcParamsDefault)
with open('/home/lambwg/ng15_rcparams.json', 'rb') as fin:
    plt.rcParams.update(json.load(fin))

In [3]:
plt.rcParams["figure.figsize"] = [3.5503666805036667, 2.1942472810764047]

In [4]:
from astroML.stats import sigmaG  # rank-reduced uncertainty

In [5]:
def bayes_fac(samples, ntol=200, min_log10rho=-9, max_log10rho=-4):
    """
    Computes the Savage Dickey Bayes Factor and uncertainty.

    :param samples: MCMCF samples of GWB (or common red noise) amplitude
    :param ntol: Tolerance on number of samples in bin

    :returns: (bayes factor, 1-sigma bayes factor uncertainty)

    """

    prior = 1 / (max_log10rho - min_log10rho)
    dA = np.linspace(0.01, 0.1, 100)
    bf = np.zeros(100)
    bf_err = np.zeros(100)
    mask = [True]*100  # selecting bins with more than 200 samples

    for ii, delta in enumerate(dA):
        n = samples[samples <= (min_log10rho + delta)].shape[0]
        N = samples.shape[0]

        post = n / N / delta

        bf[ii] = prior/post
        bf_err[ii] = bf[ii]/np.sqrt(n)

        if n < ntol:
            mask[ii] = False

    return np.mean(bf[mask]), np.std(bf[mask])

In [17]:
pulsar_list = np.loadtxt('/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_0/ceffyl_files/epa_sj_94_10k/pulsar_list.txt',
                         dtype=np.unicode_)

In [18]:
pulsar_list[20]

'J1713+0747'

# What is the effective number of pulsars in a simulated data set?

Use GFL Lite single pulsar free spectra

In [6]:
# labels for log10rho
rho_labels = [f'log10_rho_{ii}' for ii in range(10)]

Tspan = 477171786.6242733  # obs timespan
df = 1/Tspan

psrs are indexed alphabetically. Change to longest to shortest tspans

In [7]:
# longest to shortest tspans
# organising psrs from LONGEST to SHORTEST tspan
psridx = np.array([23, 15, 11,  1, 38, 28, 31, 19, 44,  0, 20,  4, 18,  6, 16,
                   33, 17, 29, 34, 12, 21, 27,  8, 26,  3, 37, 22,  2, 32, 39,
                   24,  5, 43, 35, 10, 25, 42, 40, 30, 14, 36, 13,  9,  7, 41])

load GFL Lite $\log_{10}\rho$ posteriors

In [8]:
# load log10rho posteriors for all pulsars
rho10 = np.array([co.Core(corepath=f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/psr_{ii}/chain.core')(rho_labels)[-700000:]
                  for ii in range(45)])

In [9]:
rho10.shape

(45, 700000, 10)

taking sigmaG on log10rho

In [ ]:
sigmarho = sigmaG(rho10, axis=1)

In [11]:
sdbf = np.zeros((45, 10))
for ii in tqdm(range(45)):
    for jj in range(10):
        sdbf[ii, jj] = bayes_fac(rho10[ii, :, jj])[0]

100%|██████████| 45/45 [04:45<00:00,  6.35s/it]


In [12]:
mask = sdbf < 1
srho2 = np.copy(sigmarho)
srho2[mask] = np.inf

taking the inverse squared sum and cumulatively add across frequencies (if we want investigate $N_\mathrm{eff}$ for various $N_f$)

In [13]:
inv_Srho2 = 1/srho2**2
sum_over_f = np.cumsum(inv_Srho2, axis=1)

In [14]:
# calculate Neff as a function of Np for Nf=10
Nf = 10

Nf_idx = Nf - 1  # correct for python indexing
idx = np.argsort(sum_over_f[:, Nf_idx])[::-1]
Neff_p = np.zeros(45)
for ii in range(1, 46):
    n = sum_over_f[idx[:ii], Nf_idx]
    Neff_p[ii-1] = n.sum()/n.max()

In [15]:
Neff_p

array([1.        , 1.4603994 , 1.91275178, 2.26521726, 2.60864869,
       2.91791119, 3.19465492, 3.44349337, 3.67010029, 3.89651744,
       4.09700013, 4.28435877, 4.45726128, 4.61067902, 4.7587166 ,
       4.88259073, 4.97869515, 5.07212782, 5.13138148, 5.18635646,
       5.22202259, 5.25606731, 5.28734216, 5.30498031, 5.31607117,
       5.32495401, 5.32495401, 5.32495401, 5.32495401, 5.32495401,
       5.32495401, 5.32495401, 5.32495401, 5.32495401, 5.32495401,
       5.32495401, 5.32495401, 5.32495401, 5.32495401, 5.32495401,
       5.32495401, 5.32495401, 5.32495401, 5.32495401, 5.32495401])

In [16]:
idx

array([20, 31, 28,  4, 18, 23, 11, 38,  3,  6, 44, 33, 26, 24, 16,  2, 27,
        0,  7, 17, 19, 13, 34,  8, 41,  1, 37, 42, 40,  5, 39,  9, 10, 29,
       12, 25, 14, 15, 36, 35, 21, 43, 32, 30, 22])

In [225]:
plt.plot(np.arange(1, 46), Neff_p)
plt.xlabel('Number of pulsars')
plt.ylabel('Effective number of pulsars')
plt.show();

##### Finding uncertainties on the posterior

Taking the same chain samples as used in Figures 6 & 7. Here, we must use GFL Lite single pulsars as we're adding frequencies/pulsars

In [56]:
# load paths to files of chains from GFL LITE per number of PULSARS
datadir = '/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_down_best//'

params2 = [f'{ii}psr' for ii in range(1, 46)]  # create labels for chains

# use la_forge's SlicesCore class to only load log10_A and gamma
downA = SlicesCore(slicedirs=[datadir+f'/{ii}psr/' for ii in range(1, 46)],
                   params=params2, pars2pull=['gwlog10_A'])
downg = SlicesCore(slicedirs=[datadir+f'/{ii}psr/' for ii in range(1, 46)],
                   params=params2, pars2pull=['gwgamma'])

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_down_best///45psr//chain_1.0.txt is loaded.

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_down_best///45psr//chain_1.0.txt is loaded.



In [57]:
# take sigmaG of these chains
std_log10A_npsr = sigmaG(downA(params2), axis=0)
std_gamma_npsr = sigmaG(downg(params2), axis=0)

In [58]:
std_log10A_npsr

array([0.23712018, 0.20901812, 0.15178253, 0.13729812, 0.12672664,
       0.12179363, 0.12003783, 0.11815215, 0.11998721, 0.12171078,
       0.11303283, 0.11408987, 0.11651098, 0.11507901, 0.11288472,
       0.11390959, 0.11482257, 0.1131993 , 0.11192041, 0.1093692 ,
       0.10755265, 0.10569659, 0.10929868, 0.10849006, 0.10898568,
       0.10768626, 0.10483336, 0.10778931, 0.10470081, 0.10540198,
       0.10509855, 0.1025131 , 0.10378489, 0.10320874, 0.10686255,
       0.10654405, 0.1083549 , 0.1052839 , 0.10403808, 0.1060185 ,
       0.10247349, 0.10381026, 0.1033412 , 0.10225223, 0.09723866])

## PLOT!

plot them together!

In [65]:
fig, ax2 = plt.subplots(ncols=1, sharey=True)

p = np.linspace(1, 5.4, 100)
#p = np.arange(1, 46)

ax2.scatter(Neff_p, std_log10A_npsr,
            label='$\log_{10}A$')
ax2.scatter(Neff_p, std_gamma_npsr,
            label='$\gamma$')

ax2.plot(p, 0.25/np.sqrt(p), c='grey', ls='--', alpha=0.75)
ax2.plot(p, 0.6/np.sqrt(p), c='grey', ls='--', alpha=0.75,
         )#label='$\propto 1/\sqrt{N_\mathrm{eff}}$')

#plt.xticks(np.arange(2,16,2))

ax2.set_xlabel(r'$N_\mathrm{eff}$')
ax2.set_ylabel('$\sigma_G$')
ax2.legend(markerfirst=False); ax2.grid(True)
ax2.set_ylim(0,1)

fig.savefig('../plots/neff.pdf',
            dpi=400, bbox_inches="tight")
plt.show();
plt.close();

In [39]:
idx

array([20, 31, 28,  4, 18, 23, 11, 38,  3,  6, 44, 33, 26, 24, 16,  2, 27,
        0,  7, 17, 19, 13, 34,  8, 41,  1, 37, 42, 40,  5, 39,  9, 10, 29,
       12, 25, 14, 15, 36, 35, 21, 43, 32, 30, 22])

In [308]:
fig, ax2 = plt.subplots(ncols=1, sharey=True)

#p = np.linspace(1, 14., 100)

ax2.scatter(np.sort(Neff_p), std_log10A_npsr,
            label='$\log_{10}A$')
ax2.scatter(Neff_p, std_gamma_npsr,
            label='$\gamma$')

scaleA = 0.9-p**0.6/12
scaleA[scaleA < 0.10] = 0.10

scaleg = 2.-p**0.6/6
scaleg[scaleg < 0.26] = 0.26

#ax2.plot(p, scaleA, c='grey', ls='--', alpha=0.75)
#ax2.plot(p, scaleg, c='grey', ls='--', alpha=0.75,
#         label='$\propto 1/\sqrt{N_\mathrm{eff}}$')

#plt.xticks(np.arange(2,16,2))

ax2.set_xlabel(r'$N_\mathrm{eff}$')
ax2.set_ylabel('$\sigma_G$')
ax2.legend(markerfirst=False); ax2.grid(True)
ax2.set_ylim(0,2)

fig.savefig('../plots/np.pdf',
            dpi=400, bbox_inches="tight")
plt.show();

# Apply to 12.5 yr data set!

In [21]:
# organising psrs from LONGEST to SHORTEST tspan
psridx = np.array([23, 15, 11,  1, 38, 28, 31, 19, 44,  0, 20,  4, 18,  6, 16,
                   33, 17, 29, 34, 12, 21, 27,  8, 26,  3, 37, 22,  2, 32, 39,
                   24,  5, 43, 35, 10, 25, 42, 40, 30, 14, 36, 13,  9,  7, 41])

In [22]:
psridx.shape

(45,)

In [23]:
# load parameter labels
rho_labels = [f'log10_rho_{ii}' for ii in range(5)]
Tspan_125 = 407576851.48121357
df_125 = 1/Tspan_125

In [24]:
# load log10rho posteriors
gfllite_chains = natsorted(glob.glob(f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/psr_*/chain.core'))
chains = np.array([co.Core(corepath=c)(rho_labels) for c in gfllite_chains])

min_shape = min(c.shape[0] for c in chains)
rho5 = np.dstack([c[-145000:] for c in chains]).T

In [25]:
chains = 0

In [26]:
rho5.shape

(45, 5, 145000)

In [27]:
# taking sigmaG of the posteriors
mu_Sf_125 = sigmaG(rho5, axis=2)

In [28]:
sdbf_125 = np.zeros((45, 5))
for ii in tqdm(range(45)):
    for jj in range(5):
        sdbf_125[ii, jj] = bayes_fac(rho5[ii, jj])[0]

100%|██████████| 45/45 [00:30<00:00,  1.49it/s]


In [29]:
mask = sdbf_125 < 1
mu_Sf_125_2 = np.copy(mu_Sf_125)
mu_Sf_125_2[mask] = np.inf

In [30]:
# cumulative sum over frequencies
sum_over_f_125 = np.cumsum(1/mu_Sf_125_2**2, axis=1)
sum_over_f_125.shape

(45, 5)

In [92]:
# calculating Neff as a function of Np for 5 freqs\
Nf = 5

Nf_idx = Nf - 1  # correct for python indexing
Neff_p_125 = np.zeros(45)

# index from greatest to smallest value of sum over freqs
idx = np.argsort(sum_over_f_125[:, Nf_idx], axis=0)[::-1]
for ii in range(1, 46):
    n = sum_over_f_125[idx[1:ii+1], Nf_idx]
    Neff_p_125[ii-1] = np.sum(n)/n.max()

In [93]:
Neff_p_125

array([1.        , 1.7891151 , 2.54343399, 2.83723071, 2.97545661,
       3.09434801, 3.21030846, 3.32499148, 3.41588497, 3.50072174,
       3.56958587, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857,
       3.60369857, 3.60369857, 3.60369857, 3.60369857, 3.60369857])

In [198]:
plt.plot(np.arange(1,46), Neff_p_125)
#plt.plot(np.arange(1,44), Neff_p_125_2)

plt.xlabel(r'$N_p$')
plt.ylabel(r'$N_\mathrm{eff}$')
plt.show();

In [108]:
plt.close();

##### load per psr analyses

In [94]:
downA_125, downg_125 = np.zeros((45, 145000)), np.zeros((45, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 46)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aNpsr_down_best/{ii}psr/chain.core')
    downA_125[ii-1] = c0('gwlog10_A')[-145000:]
    downg_125[ii-1] = c0('gwgamma')[-145000:]

  2%|▏         | 1/45 [00:00<00:08,  5.44it/s]

  7%|▋         | 3/45 [00:00<00:09,  4.55it/s]

  9%|▉         | 4/45 [00:00<00:10,  4.09it/s]

 13%|█▎        | 6/45 [00:01<00:07,  5.25it/s]

 18%|█▊        | 8/45 [00:01<00:06,  5.62it/s]

...

 87%|████████▋ | 39/45 [00:07<00:00,  6.30it/s]

 91%|█████████ | 41/45 [00:07<00:00,  7.35it/s]

 93%|█████████▎| 42/45 [00:08<00:00,  6.85it/s]

 98%|█████████▊| 44/45 [00:08<00:00,  6.91it/s]

100%|██████████| 45/45 [00:08<00:00,  5.29it/s]


In [79]:
c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aNpsr_down_best/2psr/chain.core')

In [95]:
# take sigmaG of these chains
std_log10A_npsr_125 = sigmaG(downA_125, axis=1)
std_gamma_npsr_125 = sigmaG(downg_125, axis=1)

In [96]:
std_log10A_npsr_125

array([0.89235643, 0.91042626, 0.57615833, 0.50622203, 0.49940848,
       0.45277247, 0.46907658, 0.45510733, 0.44475496, 0.42868816,
       0.42382088, 0.43352393, 0.43369055, 0.44436066, 0.44525375,
       0.43835805, 0.43612288, 0.43515453, 0.44535767, 0.44195699,
       0.43949073, 0.44473196, 0.4404682 , 0.44440241, 0.44461673,
       0.44034062, 0.4599379 , 0.44995622, 0.45250819, 0.45530307,
       0.44182867, 0.45301894, 0.44565437, 0.46834486, 0.4430703 ,
       0.43839972, 0.42095434, 0.41385916, 0.4055485 , 0.41498487,
       0.41202038, 0.40043632, 0.40867127, 0.41344299, 0.36693237])

In [105]:
fig, ax2 = plt.subplots(ncols=1, sharey=False)

# -----------------------------
p = np.linspace(1, 3.6, 100)

ax2.scatter(Neff_p_125, std_log10A_npsr_125,
            label='$\log_{10}A$')
ax2.scatter(Neff_p_125, std_gamma_npsr_125,
            label='$\gamma$')

ax2.plot(p, 1.75/np.sqrt(p), c='grey', ls='--', alpha=0.75,
         #label='$\propto 1/\sqrt{N_\mathrm{eff}}$')
        )
ax2.plot(p, 0.85/np.sqrt(p), c='grey', ls='--', alpha=0.75)

#plt.xticks(np.arange(2,16,2))

ax2.set_xlabel(r'$N_\mathrm{eff}$')
ax2.set_title(r'Increasing $N_p$')
ax2.set_ylabel('$\sigma_G$')
ax2.legend(markerfirst=False); ax2.grid(True)
#ax2.set_ylim(0.25,1.5)

fig.savefig('../plots/neff_125.pdf',
            dpi=400, bbox_inches="tight")
plt.show();

In [98]:
plt.close();

In [52]:
idx

array([ 0, 44,  1, 28,  4,  2, 23,  6, 16, 24, 38, 19, 41, 12, 15, 14, 13,
       42, 11, 17,  9,  8,  7,  5,  3, 10, 21, 18, 20, 40, 39, 37, 36, 35,
       34, 33, 32, 31, 30, 29, 27, 26, 25, 43, 22])